# Ch19. 동적 속성과 프로퍼티

속성 = 데이터 속성 또는 메서드

배울 것 
- 동적 속성을 이용한 json 예제 
  - `__new__()` 특별 메서드, shelve  
- 프로퍼티 
  - 프로퍼티 데코레이터, 게터, 세터, 문서화
  - 프로퍼티 팩토리 예제
- 속성 제거, 특별 속성 및 내장함수
- Unified Access Principle (UAP)

### 19.1-3 동적 속성

In [13]:
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = '/osconfeed.json'
folder_path = os.path.dirname(os.path.realpath("__file__"))
print(folder_path)
def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)
        with urlopen(URL) as remote, open(os.path.join(folder_path, JSON), 'wb') as local:
            local.write(remote.read())
            
    with open(JSON, encoding="utf-8") as fp:
        return json.load(fp)

/notebooks


In [14]:
feed = load()

In [15]:
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [16]:
for key, value in sorted(feed['Schedule'].items()):
    print('{:3} {}'.format(len(value), key))

  1 conferences
494 events
357 speakers
 53 venues


In [17]:
feed['Schedule']['speakers'][-1]['name']

'Carina C. Zona'

In [22]:
from collections import abc

class FrozenJSON:
    def __init__(self, mapping):
        self.__data = dict(mapping)
        
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON.build(self.__data[name])
        
    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:
            return obj

In [23]:
raw_feed = load()

In [24]:
feed = FrozenJSON(raw_feed)

In [25]:
len(feed.Schedule.speakers)

357

In [26]:
sorted(feed.Schedule.keys())

['conferences', 'events', 'speakers', 'venues']

In [27]:
feed.Schedule.speakers[-1].name

'Carina C. Zona'

`FrozenJSON` 클래스는 파이썬 키워드가 속성명으로 사용된 경우나 사용된 키가 올바른 파이썬 식별자가 아닌 경우를 처리하지 못한다. 

In [28]:
grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982, '2be': 'or not'})

In [29]:
grad.class

SyntaxError: invalid syntax (<ipython-input-29-bb5c99ef29c5>, line 1)

In [30]:
getattr(grad, 'class')

1982

In [29]:
x.2be

SyntaxError: invalid syntax (<ipython-input-29-8694215ab5bd>, line 1)

In [34]:
getattr(grad, '2be')

'or not'

`__new__()` 메서드는 `@classmethod`를 사용하지 않는 특별 클레스 메서드로서 반드시 객체를 반환하는 생성자이다. 그 객체가 `__init__()`의 첫번째 인수 `self`로 전달된다.

In [16]:
from collections import abc
from keyword import iskeyword

class FrozenJSON:
    def __new__(cls, arg):
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)
        elif isinstance(arg, abc.MutableSequence):
            return [cls(item) for item in arg]
        else:
            return arg
    
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if iskeyword(key):
                key += '_'
            self.__data[key] = value
        
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name])

### 19.1.4 `shelve`

`multiprocessing.Namespace`, `argparse.Namespace` 참고

In [23]:
import warnings

DB_NAME = 'data/schedule1_db'
CONFERENCE = 'conference.115'

class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
def load_db(db):
    raw_data = load()
    warnings.warn('loading '+ DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1]
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = Record(**record)

In [24]:
import shelve

In [25]:
db = shelve.open(DB_NAME)

In [26]:
if CONFERENCE not in db:
    load_db(db)

In [27]:
speaker = db['speaker.3471']

In [28]:
type(speaker)

__main__.Record

In [29]:
speaker.name, speaker.twitter

('Anna Martelli Ravenscroft', 'annaraven')

In [59]:
db.close()

## 19.2 - 3 프로퍼티

In [35]:
class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price
    
    @property
    def weight(self):
        return self.__weight
    
    @weight.setter
    def weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')

In [36]:
walnuts = LineItem('walnuts', 0, 10.00)

ValueError: value must be > 0

In [42]:
class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price
    
    def get_weight(self):
        return self.__weight
    
    def set_weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')
            
    weight = property(get_weight, set_weight)

In [43]:
walnuts = LineItem('walnuts', 0, 10.00)

ValueError: value must be > 0

*객체 속성은 클래스 데이터 속성을 가리지만 객체 속성은 클래스 프로퍼티를 가리지 않는다,*
*새로운 클래스 프로퍼티는 기존 객체 속성을 가린다*

In [32]:
class Class:
    data = 'the class data attr'
    @property
    def prop(self):
        return 'the prop value'
    
obj = Class()

In [33]:
vars(obj)

{}

In [34]:
obj.data

'the class data attr'

In [35]:
obj.data = 'bar'

In [36]:
vars(obj)

{'data': 'bar'}

In [37]:
obj.data

'bar'

In [38]:
Class.data

'the class data attr'

In [39]:
Class.prop

In [40]:
obj.prop

'the prop value'

In [41]:
obj.prop = 'foo'

AttributeError: can't set attribute

In [42]:
obj.__dict__['prop'] = 'foo'

In [43]:
vars(obj)

{'data': 'bar', 'prop': 'foo'}

In [44]:
obj.prop

'the prop value'

In [45]:
Class.prop = 'baz'
obj.prop

'foo'

In [46]:
obj.data

'bar'

In [47]:
Class.data

'the class data attr'

In [48]:
Class.data = property(lambda self: 'the "data" prop value')

In [49]:
obj.data

'the "data" prop value'

In [103]:
del Class.data
obj.data

'bar'

프로퍼티 문서화 예제

In [50]:
class Foo:
    @property
    def bar(self):
        '''The bar attribute'''
        return self.__dict__['bar']
    
    @bar.setter
    def bar(self, value):
        self.dict['bar'] = value

In [51]:
help(Foo.bar)

Help on property:

    The bar attribute



In [52]:
help(Foo)

Help on class Foo in module __main__:

class Foo(builtins.object)
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  bar
 |      The bar attribute



## 19.4 프로퍼티 팩토리

In [44]:
def quantity(storage_name):
    def qty_getter(instance):
        return instance.__dict__[storage_name]
    
    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError('value must be > 0')
            
    return property(qty_getter, qty_setter)

class LineItem:
    weight = quantity('weight')
    price = quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [45]:
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)
nutmeg.weight, nutmeg.price

(8, 13.95)

In [46]:
sorted(vars(nutmeg).items())

[('description', 'Moluccan nutmeg'), ('price', 13.95), ('weight', 8)]

## 19.5 속성 제거

In [56]:
class BlackKnight:
    def __init__(self):
        self.members = ['an arm', 'another arm', 'a leg', 'another leg']
        self.phrases = ["'Tis but a scratch",
                        "It's just a flesh wound.",
                        "I'm invincible",
                        "All right, we'll call it a draw."]
    @property
    def member(self):
        print('next meber is:')
        return self.members[0]
    
    @member.deleter
    def member(self):
        text = 'BLACK KNIGHT (loses {})\n-- {}'
        print(text.format(self.members.pop(0), self.phrases.pop(0)))
        
    # member = property(member_getter, fdel=member_deleter)

In [57]:
knight = BlackKnight()
knight.member

next meber is:


'an arm'

In [58]:
del knight.member

BLACK KNIGHT (loses an arm)
-- 'Tis but a scratch


In [59]:
del knight.member

BLACK KNIGHT (loses another arm)
-- It's just a flesh wound.


In [60]:
del knight.member

BLACK KNIGHT (loses a leg)
-- I'm invincible


In [61]:
del knight.member

BLACK KNIGHT (loses another leg)
-- All right, we'll call it a draw.


프로퍼티를 사용하지 않고 동적 속성을 사용할 때는 `__delattr__()` 을 사용하여 속성을 제거할 수 있다

## 19.6 속성을 처리하는 핵심 속성 및 함수

#### 속성

- `__class__` : 객체 클래스에 대한 참조, obj.__class__ 는 type(obj)와 동일, 파이썬은 `__getattr__()` 과 같은 특별 메서드를 객체 자체가 아니라 객체 클라스에서만 검색한다.
- `__dict__` : 객체나 클래스의 쓰기가능 속성을 저장하는 매핑. `__dict__`를 가진 객체는 임의의 새로운 속성을 언제든지 설정할 수 있다.
- `__slots__` : 허용된 속성의 이름들을 담은 튜플. `__dict__`가 `__slots__`에 들어 있지 않으면 이 클래스의 객체는 자체적인 `__dict__`를 가질 수 없고 여기에 나열된 속성만 만들 수 있다.

#### 내장함수 
- `dir([obj])` : "대부분의" 객체 속성을 나열한다.
- `getattr(object, name[, default])`, `hasattr(object, name)`, `setattr(object, name, value)`
- `vars([obj])` : obj의 `__dict__`를 반환. `__slots__`은 있고 `__dict__`는 없는 객체는 처리할 수 없다. 인수가 없으면 `locals()`와 같다.

#### 특별 메서드 
사용자 정의 클래스의 경우 객체의 `__dict__`가 아니라 객체의 클래스에 정의 되어야 암묵적으로 호출하는 특별 메서드가 제대로 작동한다.
- `__delattr__(self, name)` : `del obj.attr`는 `Class.__delattr__(obj, 'attr')`을 호출
- `__dir__(self)`: `dir(obj)` 는 `Class.__dir__(obj)`를 호출
- `__getattr__(self, name)` vs. `__getattribute__(self, name)` : 후자는 `getattr`또는 `hasattr`등 특별속성이나 메서드가 아닌 속성을 가져올 때 언제나 호출, 실패해서 `AttributeError`가 발생하면 전자 호출
- `__setattr__(self, name)` : `setattr`에 의해 호출

## 참고 읽을 거리

- https://docs.python.org/3/library/functions.html Built-in Functions
- Unified Access Principle (UAP):
  - Object-Oriented Software Construction, Betrand Meyer
  - http://wiki.c2.com/?UniformAccessPrinciple  by Ward Cunningham